# Group 6 
## Shor’s Integer Factorization

* Bijayan Ray 
* Hardik Kalra
* Hrishikesh Saikia

In [1]:
import math, random
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile, execute

# Quantum order finding function implementation
def order_finding(x, N): # quantum order finding method
    l = 2 # 2 bit number 
    e = 0.1 # precision e = 0.1 (one can increase this but time of computation will increase)
    
    t = 2 * l + 1 + math.ceil(math.log(2 + 1 / (2 * e)))
    n1 = 2 * t
    n2 = l

    # Creating quantum and classical registers 
    q1 = QuantumRegister(n1, 'q1')
    q2 = QuantumRegister(n2, 'q2')
    c = ClassicalRegister(n1, 'c')

    # Creating the quantum circuits
    qc = QuantumCircuit(q1, q2, c)

    # Creating superposition
    qc.h(q1)

    # Applying the U_{x,N} gate
    for j in range(2 ** t):
        modular_multiply_gate(qc, x, N, t, j, q1, q2) # calling the modular multiplication gate in order to exponentiate

    # Applying the Inverse Quantum Fourier Transform
    qc.swap(q1[0:t], q1[t:2*t])

    for i in range(t):
        for j in range(i): # looping through the registers 
            qc.cp(-np.pi / float(2 ** (i - j)), q1[t + j], q1[t + i])

        qc.h(q1[t + i])

    # Measuring first register
    qc.measure(q1, c)

    # Transpilation
    simulator = Aer.get_backend('aer_simulator')
    compiled_circuit = transpile(qc, simulator)
   
    result = execute(compiled_circuit, simulator).result() # storing the result in dictionary format

    # Keeping track of the possible values 
    counts = result.get_counts()
   
    # Storing the most probable element
    most_probable = max(counts, key=counts.get)
    max_frequency = counts[most_probable]

    # Find all elements with the maximum probability
    max_elements = [element for element, frequency in counts.items() if frequency == max_frequency]

    # Find the smallest element which are most probable
    smallest_most_probable = min(max_elements)
    
    # Returning the order of in decimal system from binary
    return int(str(smallest_most_probable), 2)
   
# Function for controlled modular multiplication gate
def modular_multiply_gate(qc, x, N, t, j, q1, q2):
    # Apply the controlled modular multiplication
    for i in range(t):
        angle = 2 * np.pi * (x ** (2 ** i) % N) / N # phase angle
        cu_gate(qc, angle, q1[i], q2[0]) # controlled unitary gate

# Function for controlled unitary operation that is required in modular multiplication
def cu_gate(qc, angle, control_qubit, target_qubit):
    cu_matrix = np.array([[1, 0], [0, np.exp(1j * angle)]]) # matrix for the controlled unitary gate
    qc.unitary(cu_matrix, [control_qubit, target_qubit]) # making the unitary gate out of the matrix above 

# Factoring using the quantum order finding as subroutine
def factoring_order_finding(n):
    if n%2 == 0: # basic cases
        return 2
    if power(n)!=None: # checking if it is power of the form x^y where x and y are integers 
        return power(n)
    x = random.randint(1,n-1) # random number generation
    g = math.gcd(x,n) # storing the GCD
    if g>1:
        return g # basic cases
    else:
        r = order_finding(x,n) # order finding x mod n
        if r is not None: # getting rid of degenerate cases
            if r % 2 == 0 and ((x**(r//2) + 1) % n) != 0: # checking according to Shor's algorithm 
                gx = math.gcd(x**(r//2)+1,n) 
                gy = math.gcd(x**(r//2)-1,n)
                if gx != 1: # checking GCD conditions
                    return gx
                elif gy != 1: # checking GCD conditions
                    return gy
                return factoring_order_finding(n) # try again if it doesn't work
            return factoring_order_finding(n) # try again if it doesn't work
        factoring_order_finding(n) # try again if it doesn't work

# This function is it check if the number n is a power of something that is if it is of the form x^y for some integers x and y
def power(n): 
    for b in range(2,int(math.log(n)+2)): # looping through possible cases 
        start = 0 
        end = n
        while (start <= end): # implementing binary search through 
            mid =( start + end )//2 # mid element
            pmid = mid ** b
            if pmid == n: # conditional statements 
                return mid
            elif pmid > n:
                if end != mid:
                    end = mid
                else:
                    break
            else:
                if start != mid:
                    start = mid
                else:
                    break
                    
n = int(input("Enter the number you want to factorize using Shor's algorithm: ")) # statement to accept user input
output = factoring_order_finding(n) # storing the factor outputted by Shor's algorithm (quantum)
print("A factor using Shor's algorithm is: ", output) # outputing the factor outputted by Shor's algorithm 
print()
print("The prime factors of ", n, " are: ") # outputting all the prime factors of n

n = n // output # since we have found one factor of n using Shor's algorithm we divide by that to make the number small and factorize them using classical methods

def primeFactors(n): # classical prime factorization function
   
    if n == 1: # basic case
        return

    while n % 2 == 0: # checking if the number is even or not 
        print(2)
        n = n // 2

    for i in range(3,int(math.sqrt(n))+1,2): # factoring
        while n % i == 0:
            print(i)
            n = n // i

    if n > 2: # conditional statement 
        print(n) # printing the factors

primeFactors(output) # factoring the factors outputted by Shor's algorithm
primeFactors(n) # factoring the factors of n after dividing it by the factor obtained using Shor's quantum algorithm 

# Note sometime due to probabilistic nature of the quantum algorithm it might take a while to factorize on same input

Enter the number you want to factorize using Shor's algorithm:  35


A factor using Shor's algorithm is:  5

The prime factors of  35  are: 
5
7
